# Strand Agents によるツールとしてのエージェント

「ツールとしてのエージェント」とは、AIシステムにおけるアーキテクチャパターンの一つで、特殊なAIエージェントを他のエージェントから利用可能な呼び出し可能な関数（ツール）としてラップします。これにより、以下の階層構造が形成されます。

1. 主要な「オーケストレーター」エージェントがユーザーインタラクションを処理し、どの特殊エージェントを呼び出すかを決定します。

2. 特殊化された「ツールエージェント」は、オーケストレーターから呼び出されると、ドメイン固有のタスクを実行します。

このアプローチは、マネージャーが各専門家を調整し、それぞれが独自の専門知識を持ち寄って複雑な問題を解決していくという、人間のチームのダイナミクスを模倣しています。単一のエージェントがすべてを処理しようとするのではなく、タスクは最も適切な特殊エージェントに委任されます。



## 主な利点と基本原則

「ツールとしてのエージェント」というパターンには、いくつかの利点があります。

- 関心の分離：各エージェントは特定の責任領域を持つため、システムの理解と保守が容易になります。
- 階層的な委任：オーケストレーターがどのスペシャリストを呼び出すかを決定し、明確な指揮系統を構築します。
- モジュール型アーキテクチャ：システム全体に影響を与えることなく、スペシャリストを個別に追加、削除、または変更できます。
- パフォーマンスの向上：各エージェントは、特定のタスクに合わせて最適化されたシステムプロンプトとツールを利用できます。


In [ ]:
!pip install -r requirements.txt

In [ ]:
import os

from strands import Agent, tool
from strands_tools import file_write

このモジュールでは、オーケストレーターベースのマルチエージェントワークフローを作成します。

<div style="text-align:left">
<img src="images/architecture.png" width="75%" />
</div>

また、ネストされたエージェントを作成できる `use_llm` についても説明します。

## リサーチエージェント

まず、http リクエスト ツールを使用して基本的なリサーチ アシスタントを作成しましょう。

In [ ]:
RESEARCH_ASSISTANT_PROMPT = """あなたは専門のリサーチアシスタントです。
リサーチ上の問いに対する回答として、事実に基づき、確かな情報源に基づいた情報を提供することにのみ注力してください。
可能な限り、常に情報源を引用してください。"""

In [ ]:
research_agent = Agent(
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
    # tools=[http_request]  # ここでエージェントAI検索ツールを有効にできる
)

query = "Amazon Bedrock とその機能の概要"
# エージェントを呼び出して応答を返す
response = research_agent(query)

これで、このエージェントをツールとしてラップできるようになりました。これにより、他のエージェントがエージェントと対話できるようになります。

#### エージェントをツールとして扱うためのベストプラクティス

Strandly AI で「ツールとしてのエージェント」パターンを実装する際の注意点は以下のとおりです。

1. 明確なツールドキュメント：エージェントの専門分野を説明するドキュメント文字列を作成します。
2. 焦点を絞ったシステムプロンプト：各特化エージェントがそれぞれのドメインに厳密に特化するようにします。
3. 適切な応答処理：一貫したパターンを使用して応答を抽出し、フォーマットします。
4. ツール選択ガイダンス：オーケストレーターに、各特化エージェントをいつ使用するかの明確な基準を提供します。

In [ ]:
@tool
def research_assistant(query: str) -> str:
    """
    リサーチ関連のクエリを処理し、回答します。

    引数:
      クエリ: 事実に基づく情報を必要とする研究上の質問

    戻り値:
      引用文献を含む詳細な研究回答
    """
    try:
        # Strandsエージェントを使用すると、専門のエージェントを簡単に作成可能
        research_agent = Agent(
            system_prompt=RESEARCH_ASSISTANT_PROMPT,
        )

        # エージェントを呼び出して応答を返す
        response = research_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in research assistant: {str(e)}"

ここで、ベスト プラクティスに従って、`product_recommendation_assistant`、`trip_planning_assistant`、および `orchestrator` エージェントを作成しましょう。

### 製品推奨アシスタント

In [ ]:
@tool
def product_recommendation_assistant(query: str) -> str:
    """
    適切な商品を提案することで、商品推奨クエリを処理します。

    引数:
      クエリ: ユーザーの好みに基づいた商品に関する問い合わせ

    戻り値:
      理由に基づいたパーソナライズされた商品推奨
    """
    try:
        product_agent = Agent(
            system_prompt="""あなたは専門的な商品推奨アシスタントです。
            ユーザーの好みに基づいて、パーソナライズされた商品提案を提供してください。必ず出典を明記してください。""",
        )
        # エージェントを呼び出して応答を返す
        response = product_agent(query)

        return str(response)
    except Exception as e:
        return f"Error in product recommendation: {str(e)}"

In [ ]:
product_recommendation_assistant("空飛ぶクルマに関する製品推奨事項")

### 旅行計画アシスタント

In [ ]:
@tool
def trip_planning_assistant(query: str) -> str:
    """
    旅行計画を作成し、旅行アドバイスを提供します。

    引数:
      クエリ: 目的地と希望を指定した旅行計画リクエスト

    戻り値:
      詳細な旅行計画または旅行アドバイス
    """
    try:
        travel_agent = Agent(
            system_prompt="""あなたは専門の旅行計画アシスタントです。
              ユーザーの好みに基づいて、詳細な旅行プランを作成します。""",
        )
        # エージェントを呼び出して応答を返す
        response = travel_agent(query)

        return str(response)
    except Exception as e:
        return f"Error in trip planning: {str(e)}"

### オーケストレーターエージェント

In [ ]:
# 明確なツール選択ガイダンスでオーケストレーターシステムプロンプトを定義
MAIN_SYSTEM_PROMPT = """
あなたは、専門エージェントに問い合わせをルーティングするアシスタントです。
- リサーチに関する質問や事実情報の場合 → research_assistant ツールを使用
- 商品のおすすめやショッピングアドバイスの場合 → product_recommendation_assistant ツールを使用
- 旅行の計画や旅程の場合 → trip_planning_assistant ツールを使用
- 専門知識を必要としない簡単な質問の場合 → 直接回答

ユーザーの問い合わせ内容に基づいて、常に最も適切なツールを選択してください。
"""

In [ ]:
# Strands Agentsはエージェントツールの簡単な統合を可能にする
orchestrator = Agent(
    system_prompt=MAIN_SYSTEM_PROMPT,
    tools=[
        research_assistant,
        product_recommendation_assistant,
        trip_planning_assistant,
        file_write,
    ],
)

In [ ]:
# 例: 電子商取引顧客サービスシステム
customer_query = (
    "ハイキングブーツを探しています。現在のディレクトリに最終的な回答を書き込みます。"
)

os.environ["BYPASS_TOOL_CONSENT"] = "true"

# オーケストレーターは複数の専門エージェントが必要であることを自動的に判断する
response = orchestrator(customer_query)

オーケストレーターのメッセージを見てみましょう。エージェントがサブエージェントをツールとして使うことを決定したことがわかります。

In [ ]:
orchestrator.messages

In [ ]:
customer_query = "パタゴニアへの旅行の計画を手伝ってもらえますか？"

response = orchestrator(customer_query)

In [ ]:
orchestrator.messages

### 複数のエージェントを呼び出す

In [ ]:
orchestrator.messages = []

In [ ]:
query = "スペインについて調べてもらえますか？7日間の旅行の計画も手伝ってもらえますか？"

orchestrator(query)

舞台裏では、オーケストレーターが以下の処理を行います。
1. まず `research_assistant` を呼び出します。
2. 次に `trip_planning_assistant` を呼び出します。
3. これらの特殊な応答を統合し、両方のクエリに対応するまとまりのある回答を作成します。

### シーケンシャルエージェント通信パターン


エージェントツールは複数のエージェントを組み合わせることもできます。この例では、`research_agent` の出力を `summary_agent` に渡し、要約されたレスポンスを返します。

In [ ]:
# ユーザークエリを定義
topic = "generative Ai"
# リサーチエージェントを作成
research_agent = Agent(
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
)
# 要約エージェントを作成
summary_agent = Agent(
    system_prompt="""
    あなたは、複雑な情報を明確かつ簡潔な要約にまとめることに重点を置く要約スペシャリストです。
    主な目標は、詳細な情報から要点、主要な議論、そして重要なデータを抽出することです。
    元のコンテンツの正確性を維持しながら、より理解しやすいものにする必要があります。
    明瞭性、簡潔性、そして情報の最も重要な側面を強調することに重点を置いてください。
    """,
)

print("Multiple agents created successfully!")
print(f"\n🔍 RESEARCH AGENT working on: {topic}\n") 
try:
    # エージェント1: リサーチエージェントを呼び出す
    research_response = research_agent(
        f"次のトピックについて包括的な情報を収集してください: {topic}."
    )
    research_text = research_response.message.content[0]["text"]
    print("\n✂️ SUMMARY AGENT distilling the research\n")
    
    # エージェント2: 要約エージェントに簡潔な要約を作成するよう依頼
    summary_response = summary_agent(
        f"次のリサーチの簡潔な要約を作成してください: {research_text}"
    )
    summary_text = summary_response.message.content[0]["text"]
    
    print(summary_text)
except Exception as e:
    print(f"Error in research assistant: {str(e)}")

## おめでとうございます！

Strands Agents でエージェントをツールとして使用し、より複雑なエージェントアプリケーションを作成する方法を学びました。